In [90]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from google.colab import files

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer


# Leyendo DataSet

In [91]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [92]:
df = pd.read_csv( "/content/drive/MyDrive/Orga Datos/TP2/Training.csv" )
df

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,Source,Billing_Country,Account_Name,Opportunity_Name,Opportunity_ID,Sales_Contract_No,Account_Owner,Opportunity_Owner,Account_Type,Opportunity_Type,Quote_Type,Delivery_Terms,Opportunity_Created_Date,Brand,Product_Type,Size,Product_Category_B,Price,Currency,Last_Activity,Quote_Expiry_Date,Last_Modified_Date,Last_Modified_By,Product_Family,Product_Name,ASP_Currency,ASP,ASP_(converted)_Currency,ASP_(converted),Planned_Delivery_Start_Date,Planned_Delivery_End_Date,Month,Delivery_Quarter,Delivery_Year,Actual_Delivery_Date,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage,Prod_Category_A
0,27761,EMEA,None,1,1,1,1,0,Bureaucratic_Code_4,6/16/2015,None,Netherlands,Account_Name_619,Opportunity_Name_12598,0,None,Person_Name_51,Person_Name_18,Account_Type_2,Opportunity_Type_1,Non Binding,Delivery_Terms_2,12/7/2015,None,None,None,None,None,None,NaT,1/30/2016,6/13/2016,Person_Name_18,Product_Family_77,Product_Name_99,EUR,0.5200,USD,0.58817,5/1/2016,6/30/2016,2016 - 5,Q2,2016,NaT,10,EUR,5272800.00,EUR,5272800.0,Closed Lost,Prod_Category_A_None
1,27760,EMEA,None,0,0,0,0,0,Bureaucratic_Code_4,6/16/2015,None,Netherlands,Account_Name_619,Opportunity_Name_12600,1,39,Person_Name_51,Person_Name_20,Account_Type_2,Opportunity_Type_1,Non Binding,Delivery_Terms_2,12/7/2015,None,None,None,None,None,None,NaT,1/20/2016,1/15/2016,Person_Name_20,Product_Family_77,Product_Name_100,EUR,0.5300,USD,0.59948,1/18/2016,1/20/2016,2016 - 1,Q1,2016,NaT,0,EUR,48230.00,EUR,48230.0,Closed Won,Prod_Category_A_None
2,27446,Americas,NW America,0,0,0,0,0,Bureaucratic_Code_4,4/21/2015,Source_7,United States,Account_Name_1794,Opportunity_Name_469,2,760,Person_Name_64,Person_Name_8,Account_Type_5,Opportunity_Type_1,Non Binding,Delivery_Terms_4,12/8/2015,None,None,None,None,None,None,NaT,9/30/2016,9/29/2016,Person_Name_8,Product_Family_81,Product_Name_91,USD,0.4800,USD,0.48000,1/25/2016,1/25/2016,2016 - 1,Q1,2016,NaT,0,USD,83865.60,USD,83865.6,Closed Won,Prod_Category_A_None
3,16808,Americas,NW America,1,0,1,0,0,Bureaucratic_Code_5,7/27/2013,Source_11,United States,Account_Name_1201,Opportunity_Name_415,3,None,Person_Name_8,Person_Name_8,Account_Type_5,Opportunity_Type_19,Non Binding,Delivery_Terms_1,12/8/2015,Other,Product_Type_0,Size_4,Product_Category_B_16,0.42,USD,NaT,NaT,3/27/2018,Person_Name_8,Product_Family_209,Product_Name_432,USD,0.5300,USD,0.53000,2/1/2018,3/31/2018,2018 - 2,Q1,2018,NaT,14,USD,7421881.50,USD,7421881.5,Closed Lost,Prod_Category_A_None
4,16805,Americas,NW America,1,0,1,0,0,Bureaucratic_Code_5,7/27/2013,Source_11,United States,Account_Name_1201,Opportunity_Name_851,4,None,Person_Name_8,Person_Name_8,Account_Type_5,Opportunity_Type_19,Non Binding,Delivery_Terms_1,12/8/2015,Other,Product_Type_0,Size_4,Product_Category_B_16,0.42,USD,NaT,6/2/2017,3/27/2018,Person_Name_8,Product_Family_209,Product_Name_432,USD,0.5300,USD,0.53000,2/1/2018,2/28/2018,2018 - 2,Q1,2018,NaT,25,USD,13357192.50,USD,13357192.5,Closed Lost,Prod_Category_A_None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16942,8781,EMEA,Austria,1,1,1,1,0,Bureaucratic_Code_4,1/15/2016,Source_7,Austria,Account_Name_726,Opportunity_Name_9245,12801,44,Person_Name_13,Person_Name_13,Account_Type_5,Opportunity_Type_1,Non Binding,Delivery_Terms_2,12/4/2015,None,None,None,None,None,None,NaT,NaT,1/19/2016,Person_Name_13,Product_Family_85,Product_Name_111,EUR,0.5200,USD,0.58817,3/21/2016,3/25/2016,2016 - 3,Q1,2016,NaT,0,EUR,103350.00,EUR,299715.0,Closed Won,Prod_Category_A_None
16943,8786,EMEA,Austria,1,1,1,1,0,Bureaucratic_Code_4,1/15/2016,Source_7,Austria,Account_Name_726,Opportunity_Name_9245

In [93]:
test = pd.read_csv( "/content/drive/MyDrive/Orga Datos/TP2/Test_TP2_Datos_2020-2C.csv" )
test

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,Source,Billing_Country,Account_Name,Opportunity_Name,Opportunity_ID,Sales_Contract_No,Account_Owner,Opportunity_Owner,Account_Type,Opportunity_Type,Quote_Type,Delivery_Terms,Opportunity_Created_Date,Brand,Product_Type,Size,Product_Category_B,Price,Currency,Last_Activity,Quote_Expiry_Date,Last_Modified_Date,Last_Modified_By,Product_Family,Product_Name,ASP_Currency,ASP,ASP_(converted)_Currency,ASP_(converted),Planned_Delivery_Start_Date,Planned_Delivery_End_Date,Month,Delivery_Quarter,Delivery_Year,Actual_Delivery_Date,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Prod_Category_A
0,6140,EMEA,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,Source_7,Germany,Account_Name_978,Opportunity_Name_6118,10689,6436,Person_Name_13,Person_Name_13,Account_Type_0,Opportunity_Type_8,Non Binding,Delivery_Terms_2,4/12/2019,None,None,None,None,None,None,NaT,4/30/2019,4/12/2019,Person_Name_13,Product_Family_171,Product_Name_332,EUR,0.325,USD,0.36761,5/1/2019,5/31/2019,2019 - 5,Q2,2019,NaT,0,EUR,162240.0,EUR,367419.0,Prod_Category_A_None
1,6146,EMEA,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,Source_7,Germany,Account_Name_978,Opportunity_Name_6118,10689,6436,Person_Name_13,Person_Name_13,Account_Type_0,Opportunity_Type_8,Non Binding,Delivery_Terms_2,4/12/2019,None,None,None,None,None,None,NaT,4/30/2019,4/12/2019,Person_Name_13,Product_Family_167,Product_Name_315,EUR,0.320,USD,0.36195,5/1/2019,5/31/2019,2019 - 5,Q2,2019,NaT,0,EUR,78624.0,EUR,367419.0,Prod_Category_A_None
2,6151,EMEA,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,Source_7,Germany,Account_Name_978,Opportunity_Name_6118,10689,6436,Person_Name_13,Person_Name_13,Account_Type_0,Opportunity_Type_8,Non Binding,Delivery_Terms_2,4/12/2019,None,None,None,None,None,None,NaT,4/30/2019,4/12/2019,Person_Name_13,Product_Family_140,Product_Name_241,EUR,0.275,USD,0.31105,5/1/2019,5/31/2019,2019 - 5,Q2,2019,NaT,0,EUR,126555.0,EUR,367419.0,Prod_Category_A_None
3,6118,EMEA,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,Source_7,Germany,Account_Name_978,Opportunity_Name_6117,10690,6504,Person_Name_13,Person_Name_13,Account_Type_0,Opportunity_Type_8,Non Binding,Delivery_Terms_2,4/12/2019,None,None,None,None,None,None,NaT,4/30/2019,4/25/2019,Person_Name_13,Product_Family_171,Product_Name_328,EUR,0.325,USD,0.36761,6/1/2019,6/28/2019,2019 - 6,Q2,2019,NaT,1,EUR,243360.0,EUR,757783.5,Prod_Category_A_None
4,6124,EMEA,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,Source_7,Germany,Account_Name_978,Opportunity_Name_6117,10690,6504,Person_Name_13,Person_Name_13,Account_Type_0,Opportunity_Type_8,Non Binding,Delivery_Terms_2,4/12/2019,None,None,None,None,None,None,NaT,4/30/2019,4/25/2019,Person_Name_13,Product_Family_167,Product_Name_315,EUR,0.320,USD,0.36195,6/1/2019,6/29/2019,2019 - 6,Q2,2019,NaT,0,EUR,157248.0,EUR,757783.5,Prod_Category_A_None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2546,16345,EMEA,KSA,1,1,0,0,0,Bureaucratic_Code_4,6/12/2017,Source_3,South Africa,Account_Name_1686,Opportunity_Name_11120,12364,6508,Person_Name_13,Person_Name_13,Account_Type_0,Opportunity_Type_1,Non Binding,Delivery_Terms_7,4/26/2019,None,None,None,None,None,None,NaT,5/24/2019,4/26/2019,Person_Name_13,Product_Family_213,Product_Name_447,USD,0.300,USD,0.30000,5/1/2019,5/10/2019,2019 - 5,Q2,2019,NaT,0,USD,124740.0,USD,147750.0,Prod_Category_A_None
2547,15218,Americas,SE America,1,1,0,0,0,Bureaucratic_Code_4,6/8/2018,None,United States,Account_Name_768,Opportunity_Name_9694,12365,6510,Person_Name_38,Person_Name_38,Account_Type_2,Opportunity_Type_1,Non Binding,Delivery_Terms_4,4/26/2019,None,None,None,None,None,None,N

# Viendo opportunity id

In [94]:
df[['TRF','Bureaucratic_Code']].value_counts()


TRF  Bureaucratic_Code  
0    Bureaucratic_Code_4    9417
     Bureaucratic_Code_5    2298
1    Bureaucratic_Code_4    1521
     Bureaucratic_Code_5     566
2    Bureaucratic_Code_4     419
                            ... 
10   Bureaucratic_Code_1       1
75   Bureaucratic_Code_4       1
73   Bureaucratic_Code_4       1
43   Bureaucratic_Code_4       1
500  Bureaucratic_Code_4       1
Length: 218, dtype: int64

# Pre proceso de datos

## Conversion a Dolar

In [95]:
df.loc[df['Total_Amount_Currency'] == 'JPY', 'Total_Amount'] = df['Total_Amount']*0.0096
df.loc[df['Total_Amount_Currency'] == 'JPY', 'Total_Amount_Currency'] = 'USD'

df.loc[df['Total_Amount_Currency'] == 'EUR', 'Total_Amount'] = df['Total_Amount']*1.17
df.loc[df['Total_Amount_Currency'] == 'EUR', 'Total_Amount_Currency'] = 'USD'

df.loc[df['Total_Amount_Currency'] == 'AUD', 'Total_Amount'] = df['Total_Amount']*0.70
df.loc[df['Total_Amount_Currency'] == 'AUD', 'Total_Amount_Currency'] = 'USD'

df.loc[df['Total_Amount_Currency'] == 'GBP', 'Total_Amount'] = df['Total_Amount']*1.29
df.loc[df['Total_Amount_Currency'] == 'GBP', 'Total_Amount_Currency'] = 'USD'

In [96]:
short_df = df[['Region', 'Delivery_Quarter','Stage', 'Total_Amount', 'Bureaucratic_Code', 'TRF']].rename(columns={'Stage': 'Decision'})
short_df = short_df[ (short_df['Decision'] == 'Closed Won') | (short_df['Decision'] == 'Closed Lost') ]
short_df['Decision'] = np.where(short_df['Decision'] == 'Closed Won',1,0)
short_df.dropna(inplace=True)

In [97]:
test.loc[test['Total_Amount_Currency'] == 'JPY', 'Total_Amount'] = test['Total_Amount']*0.0096
test.loc[test['Total_Amount_Currency'] == 'JPY', 'Total_Amount_Currency'] = 'USD'

test.loc[test['Total_Amount_Currency'] == 'EUR', 'Total_Amount'] = test['Total_Amount']*1.17
test.loc[test['Total_Amount_Currency'] == 'EUR', 'Total_Amount_Currency'] = 'USD'

test.loc[test['Total_Amount_Currency'] == 'AUD', 'Total_Amount'] = test['Total_Amount']*0.70
test.loc[test['Total_Amount_Currency'] == 'AUD', 'Total_Amount_Currency'] = 'USD'

test.loc[test['Total_Amount_Currency'] == 'GBP', 'Total_Amount'] = test['Total_Amount']*1.29
test.loc[test['Total_Amount_Currency'] == 'GBP', 'Total_Amount_Currency'] = 'USD'

In [98]:
short_test = test[['Region', 'Delivery_Quarter', 'Total_Amount', 'Bureaucratic_Code', 'TRF']]
short_test.dropna(inplace=True)
#\.rename(columns={'Stage': 'Decision'})
#short_test['Decision'] = np.where(short_test['Decision'] == 'Closed Won',1,0)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


## Spliteando

In [99]:
X = short_df.drop( 'Decision', axis=1 )
y = short_df['Decision']
X

,Region,Delivery_Quarter,Total_Amount,Bureaucratic_Code,TRF
0,EMEA,Q2,6169176.00,Bureaucratic_Code_4,10
1,EMEA,Q1,56429.10,Bureaucratic_Code_4,0
2,Americas,Q1,83865.60,Bureaucratic_Code_4,0
3,Americas,Q1,7421881.50,Bureaucratic_Code_5,14
4,Americas,Q1,13357192.50,Bureaucratic_Code_5,25
...,...,...,...,...,...
16942,EMEA,Q1,120919.50,Bureaucratic_Code_4,0
16943,EMEA,Q2,108827.55,Bureaucratic_Code_4,0
16944,EMEA,Q1,120919.50,Bureaucratic_Code_4,0
16945,Americas,Q2,2346796.88,Bureaucratic_Code_4,4


In [100]:
X['Total_Amount_Above_average'] = X['Total_Amount'] > X['Total_Amount'].mean()
X['TRF_Above_average'] = X['TRF'] > X['TRF'].mean()
X.drop(columns=['TRF', 'Total_Amount'], inplace=True)
X

,Region,Delivery_Quarter,Bureaucratic_Code,Total_Amount_Above_average,TRF_Above_average
0,EMEA,Q2,Bureaucratic_Code_4,True,True
1,EMEA,Q1,Bureaucratic_Code_4,False,False
2,Americas,Q1,Bureaucratic_Code_4,False,False
3,Americas,Q1,Bureaucratic_Code_5,True,True
4,Americas,Q1,Bureaucratic_Code_5,True,True
...,...,...,...,...,...
16942,EMEA,Q1,Bureaucratic_Code_4,False,False
16943,EMEA,Q2,Bureaucratic_Code_4,False,False
16944,EMEA,Q1,Bureaucratic_Code_4,False,False
16945,Americas,Q2,Bureaucratic_Code_4,True,True


In [101]:
short_test['Total_Amount_Above_average'] = short_test['Total_Amount'] > short_test['Total_Amount'].mean()
short_test['TRF_Above_average'] = short_test['TRF'] > short_test['TRF'].mean()
short_test.drop(columns=['TRF', 'Total_Amount'], inplace=True)
short_test

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/us

,Region,Delivery_Quarter,Bureaucratic_Code,Total_Amount_Above_average,TRF_Above_average
0,EMEA,Q2,Bureaucratic_Code_4,False,False
1,EMEA,Q2,Bureaucratic_Code_4,False,False
2,EMEA,Q2,Bureaucratic_Code_4,False,False
3,EMEA,Q2,Bureaucratic_Code_4,False,False
4,EMEA,Q2,Bureaucratic_Code_4,False,False
...,...,...,...,...,...
2546,EMEA,Q2,Bureaucratic_Code_4,False,False
2547,Americas,Q4,Bureaucratic_Code_4,False,False
2548,Americas,Q4,Bureaucratic_Code_4,False,False
2549,Americas,Q3,Bureaucratic_Code_4,False,False


In [102]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

In [103]:
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
X_train.shape

(5048, 5)
(11776,)
(5048,)


(11776, 5)

## Codificado

In [104]:
ohe_1 = OneHotEncoder()
ohe_1.fit(X_train)
ohe_2 = OneHotEncoder()
ohe_2.fit(short_test)



X_train_enc = ohe_1.transform(X_train)
#encoded = ohe_1.transform(X_train).toarray()
X_test_enc = ohe_1.transform(X_test)
short_test_enc = ohe_1.transform( short_test )

In [105]:
rfc = RandomForestClassifier(n_estimators=100, max_depth=6, criterion='gini' )
#n_estimators se refiere a la cantidad de arboles de decisiones que usa el rfc

In [106]:
rfc.fit(X_train_enc, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=6, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [107]:
rfc_pred = rfc.predict(X_test_enc)

In [108]:
rfc_pred

array([1, 1, 0, ..., 1, 1, 1])

In [109]:
print(classification_report(y_test, rfc_pred))
print(confusion_matrix(y_test, rfc_pred))

              precision    recall  f1-score   support

           0       0.81      0.53      0.64      2179
           1       0.72      0.91      0.80      2869

    accuracy                           0.74      5048
   macro avg       0.76      0.72      0.72      5048
weighted avg       0.76      0.74      0.73      5048

[[1151 1028]
 [ 272 2597]]


# Prediciendo Test

In [110]:
print(short_test_enc)

  (0, 2)	1.0
  (0, 6)	1.0
  (0, 13)	1.0
  (0, 15)	1.0
  (0, 17)	1.0
  (1, 2)	1.0
  (1, 6)	1.0
  (1, 13)	1.0
  (1, 15)	1.0
  (1, 17)	1.0
  (2, 2)	1.0
  (2, 6)	1.0
  (2, 13)	1.0
  (2, 15)	1.0
  (2, 17)	1.0
  (3, 2)	1.0
  (3, 6)	1.0
  (3, 13)	1.0
  (3, 15)	1.0
  (3, 17)	1.0
  (4, 2)	1.0
  (4, 6)	1.0
  (4, 13)	1.0
  (4, 15)	1.0
  (4, 17)	1.0
  :	:
  (2546, 2)	1.0
  (2546, 6)	1.0
  (2546, 13)	1.0
  (2546, 15)	1.0
  (2546, 17)	1.0
  (2547, 1)	1.0
  (2547, 8)	1.0
  (2547, 13)	1.0
  (2547, 15)	1.0
  (2547, 17)	1.0
  (2548, 1)	1.0
  (2548, 8)	1.0
  (2548, 13)	1.0
  (2548, 15)	1.0
  (2548, 17)	1.0
  (2549, 1)	1.0
  (2549, 7)	1.0
  (2549, 13)	1.0
  (2549, 15)	1.0
  (2549, 17)	1.0
  (2550, 2)	1.0
  (2550, 8)	1.0
  (2550, 14)	1.0
  (2550, 16)	1.0
  (2550, 18)	1.0


In [111]:
pred_test = rfc.predict( short_test_enc )
pred_test

array([1, 1, 1, ..., 1, 1, 0])

In [112]:
resultado = pd.DataFrame()
resultado['Opportunity_ID'] = test['Opportunity_ID']
resultado['target'] = pred_test
resultado.sort_values('Opportunity_ID',inplace = True)
resultado = resultado.drop_duplicates('Opportunity_ID',keep = 'last')
resultado.set_index('Opportunity_ID', inplace = True)
resultado.value_counts()

target
1         1302
0          265
dtype: int64

In [113]:
csv3 = resultado.to_csv('RFC_pred_1.csv')
#files.download('RFC_pred_1.csv')